In [ ]:
import subprocess
import configparser
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import os
from analysis_utils import *

if not os.path.isdir('./graphs'):
    os.makedirs('./graphs')

import math

%load_ext autoreload
%autoreload 2

# Fundamentalists with 1 bottom-feeder

In [ ]:
iterations = 100

In [ ]:
for i in range(1, 4):
    print(i)
    config = configparser.ConfigParser()
    config['fundamental'] = {'source': 'fundamentalist',
                             'n':i,
                             'cash': 350,
                             'params':'{\'r\':1.5}'}
    prey = [k for k in range(0,i)]
    config['bottomfeeder'] = {'source': 'bottomfeeder_v2',
                              'n':1,
                              'cash': 350,
                              'params':{'prey': prey, 'ordersno': 4}}
    config['noisy'] = {'source': 'noise',
                       'n':4-i-1,
                       'cash': 350,
                       'params': '{\'var\': 1, \'horizon\': 50}'}
    with open('configs/temp_config.ini', 'w') as configfile:
        config.write(configfile)
    for j in tqdm(range(0, iterations)):
        subprocess.run(['./market-des', '-r figgie -s 10000', '--index '+str(j),
               '--output '+'bvsf.'+str(i), '--config '+"temp_config"],stdout=subprocess.DEVNULL)


In [ ]:
for i in range(1, 4):
    filename = 'bvsf'+'.'+str(i)
    print(i)
    plot_rewards(filename, iterations, filename)

# Fundamentalists vs Noise Traders

In [ ]:
configs = ['4n','1f3n','2f2n','3f1n','4f']
iterations = 100

In [ ]:
for config in configs:
    print(config)
    for i in tqdm(range(0, iterations)):
        subprocess.run(['./market-des', '-r figgie -s 10000', '--index '+str(i),
                       '--output '+config, '--config '+config],stdout=subprocess.DEVNULL)

In [ ]:
for config in configs:
    print(config)
    plot_rewards(config, iterations, config)

In [ ]:
filename='1f3n.1'
plot_cashes(filename)
data = load_data(filename)
agents = data['agents']
print([agent.rate_c for agent in agents])

In [ ]:
plot_expectations('1f3n.1')

In [ ]:
plot_orders('1f3n.1',1,0)

In [ ]:
plot_prices('1f3n.1')

# Degrees of (bad) bottom-feeding

In [ ]:
iterations=100
config = 'bottomfeeders'
for i in tqdm(range(0, iterations)):
    subprocess.run(['./market-des', '-r figgie -s 10000', '--index '+str(i),
                   '--output '+config, '--config '+config],stdout=subprocess.DEVNULL)

In [ ]:
plot_rewards('bottomfeeders',iterations,"bottomfeeders")

# Degrees of (good) bottom-feeding

In [ ]:
iterations=100
config = 'bottomfeeders_v2'
for i in tqdm(range(0, iterations)):
    subprocess.run(['./market-des', '-r figgie -s 10000', '--index '+str(i),
                   '--output '+config, '--config '+config],stdout=subprocess.DEVNULL)

In [ ]:
plot_rewards('bottomfeeders_v2',iterations,"bottomfeeders_v2")

In [ ]:
data = load_data('bottomfeeders_v2.2')
cashes = data['agents'][0].cashes
plt.scatter(cashes[:,0],cashes[:,1])
plt.show()

In [ ]:
plot_expectations('bottomfeeders_v2.1')

In [ ]:
data = load_data('bottomfeeders_v2.1')
orders = data['agents'][1].orders
orders = [order for order in orders if order.assetno == 3]
order_prices = [order.price for order in orders]
colors = ["red" if order.buy else "green" for order in orders]
plt.scatter(range(0,len(orders)),order_prices,color = colors)
plt.show()

In [ ]:
plot_expectations('3f1n.0')

In [ ]:
# multiple first-degrees looking at fundamentalist

In [ ]:
iterations=100
for i in tqdm(range(0, iterations)):
    subprocess.run(['./market-des', '-r figgie -s 10000', '--index '+str(i),
                   '--output '+'speeds', '--config '+'speeds'],stdout=subprocess.DEVNULL)

In [ ]:
plot_rewards('speeds',iterations,'speeds')
# try to do it with 1 agent at latency 1 and 3 at latency 0

In [ ]:
plot_expectations('speeds.0')

In [ ]:
from statsmodels.tsa.stattools import acf

In [ ]:
nlags = 40
acfs = np.zeros(nlags+1)
iterations=20
for i in range(0, iterations):
    data = load_data('speeds.'+str(i))
    price_series = data['assets'][1].price_series[:,1]
    return_series = price_series[1:]/price_series[:-1]
    if len(return_series) == 0:
        continue
    acfs += acf(return_series, fft=True, nlags=nlags)/iterations
    #plt.plot(acf(return_series,fft=True,nlags=40),color='black',alpha=0.1)
plt.plot(acfs)
plt.show()